In [1]:
import nltk

nltk.download('punkt')
nltk.download('wordnet')
import random
import numpy as np
import json
import pickle
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model
lemmatizer=WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [2]:
with open(r'/content/drive/MyDrive/Colab Notebooks/intents (1).json') as json_file:
    intents = json.load(json_file)

In [3]:
words=pickle.load(open(r'/content/drive/MyDrive/Colab Notebooks/words4.pkl','rb'))
classes=pickle.load(open(r'/content/drive/MyDrive/Colab Notebooks/classes4.pkl','rb'))
model=load_model(r'/content/drive/MyDrive/Colab Notebooks/Dataset/medchatbot_lstm.h5')

In [4]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
  sentence_words=clean_up_sentence(sentence)
  bag=[0]*len(words)
  for w in sentence_words:
    for i,word in enumerate(words):
      if word == w:
        bag[i]=1
  return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    if not results:
        # If no intents are predicted with high confidence, return a default intent
        return_list.append({'intent': 'unknown', 'probability': '1.0'})
    else:
        for r in results:
            return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list

def get_response(intents_list, intents_json):
    tag = 'unknown'  # Default tag
    if intents_list:
        tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            return result
    return "I'm sorry, I didn't understand that. Can you please rephrase?"

print("Hi! I am MedBot. How can I help you?")

while True:
  message=input("")
  ints=predict_class(message)
  res=get_response(ints,intents)
  print(res)
  if message.lower() == 'bye':
        print("Chatbot: Goodbye! Have a great day.")
        break

Hi! I am MedBot. How can I help you?
hi
1/1 [==============================] - 1s 567ms/step
I'm sorry, I didn't understand that. Can you please rephrase?
hi
1/1 [==============================] - 0s 39ms/step
I'm sorry, I didn't understand that. Can you please rephrase?
ok
1/1 [==============================] - 0s 185ms/step
I'm sorry, I didn't understand that. Can you please rephrase?
bye
1/1 [==============================] - 0s 41ms/step
I'm sorry, I didn't understand that. Can you please rephrase?
Chatbot: Goodbye! Have a great day.
